In [1]:
import utils.utils as utils
import torch
from torch.optim import LBFGS
from torch.autograd import Variable
import numpy as np
import os
import cv2



In [2]:
def build_loss(neural_net, optimizing_img, target_representations, content_feature_maps_index, style_feature_maps_indices, config):
    target_content_representation = target_representations[0]
    target_style_representation = target_representations[1]

    current_set_of_feature_maps = neural_net(optimizing_img)

    current_content_representation = current_set_of_feature_maps[content_feature_maps_index].squeeze(axis=0)
    content_loss = torch.nn.MSELoss(reduction='mean')(target_content_representation, current_content_representation)

    style_loss = 0.0
    current_style_representation = [utils.gram_matrix(x) for cnt, x in enumerate(current_set_of_feature_maps) if cnt in style_feature_maps_indices]
    for gram_gt, gram_hat in zip(target_style_representation, current_style_representation):
        style_loss += torch.nn.MSELoss(reduction='sum')(gram_gt[0], gram_hat[0])
    style_loss /= len(target_style_representation)

    tv_loss = utils.total_variation(optimizing_img)

    total_loss = config['content_weight'] * content_loss + config['style_weight'] * style_loss + config['tv_weight'] * tv_loss

    return total_loss, content_loss, style_loss, tv_loss

In [3]:
def make_tuning_step(neural_net, optimizer, target_representations, content_feature_maps_index, style_feature_maps_indices, config):
    # Builds function that performs a step in the tuning loop
    def tuning_step(optimizing_img):
        total_loss, content_loss, style_loss, tv_loss = build_loss(neural_net, optimizing_img, target_representations, content_feature_maps_index, style_feature_maps_indices, config)
        # Computes gradients
        total_loss.backward()
        # Updates parameters and zeroes gradients
        optimizer.step()
        optimizer.zero_grad()
        return total_loss, content_loss, style_loss, tv_loss

    # Returns the function that will be called inside the tuning loop
    return tuning_step

In [4]:
def neural_style_transfer(config):
    content_images_dir = os.path.join(os.path.abspath('data') , 'content-images')
    style_images_dir = os.path.join(os.path.abspath('data'), 'style-images')
    output_img_dir = os.path.join(os.path.abspath('data'), 'output-images')
    img_format = (4, '.jpg')  # saves images in the format: %04d.jpg

    content_img_path = os.path.join(content_images_dir, config['content_img_name'])
    style_img_path = os.path.join(style_images_dir,config['style_img_name'] )

    out_dir_name = 'combined_' + os.path.split(content_img_path)[1].split('.')[0] + '_' + os.path.split(style_img_path)[1].split('.')[0]
    dump_path = os.path.join(output_img_dir, out_dir_name)
    os.makedirs(dump_path, exist_ok=True)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    content_img = utils.prepare_img(content_img_path, 400, device)
    style_img = utils.prepare_img(style_img_path, 400, device)

    init_img = content_img
    optimizing_img = Variable(init_img, requires_grad=True)

    neural_net, content_feature_maps_index_name, style_feature_maps_indices_names = utils.prepare_model('vgg19', device)
    print(f'Using vgg19 in the optimization procedure.')

    content_img_set_of_feature_maps = neural_net(content_img)
    style_img_set_of_feature_maps = neural_net(style_img)

    target_content_representation = content_img_set_of_feature_maps[content_feature_maps_index_name[0]].squeeze(axis=0)
    target_style_representation = [utils.gram_matrix(x) for cnt, x in enumerate(style_img_set_of_feature_maps) if cnt in style_feature_maps_indices_names[0]]
    target_representations = [target_content_representation, target_style_representation]

    num_of_iterations_lbfgs = 1000

    optimizer = LBFGS((optimizing_img,), max_iter=num_of_iterations_lbfgs, line_search_fn='strong_wolfe')
    cnt = 0

    def closure():
        nonlocal cnt
        if torch.is_grad_enabled():
            optimizer.zero_grad()
        total_loss, content_loss, style_loss, tv_loss = build_loss(neural_net, optimizing_img, target_representations, content_feature_maps_index_name[0], style_feature_maps_indices_names[0], config)
        if total_loss.requires_grad:
            total_loss.backward()
        with torch.no_grad():
            print(f'L-BFGS | iteration: {cnt:03}, total loss={total_loss.item():12.4f}, content_loss={config["content_weight"] * content_loss.item():12.4f}, style loss={config["style_weight"] * style_loss.item():12.4f}, tv loss={config["tv_weight"] * tv_loss.item():12.4f}')
            utils.save_and_maybe_display(optimizing_img, dump_path, config, cnt, num_of_iterations_lbfgs, should_display=False)

        cnt += 1
        return total_loss

    optimizer.step(closure)

    return dump_path

In [5]:
if __name__ == "__main__":
    resource_path=os.path.abspath('data')
    content_image_dir=os.path.join(resource_path,'content-images')
    content_image=os.path.join(content_image_dir,'figures.jpg')
    style_image_dir=os.path.join(resource_path,'style-images')
    style_image=os.path.join(style_image_dir,'vg_starry_night.jpg')
    output_image_dir=os.path.join(resource_path, 'output-images')
    
    optimization_config = {
        'img_format': (4, '.jpg'),
        'content_img_name': 'content_image_2.jpg',
        'style_img_name': 'style_example2.jpg',
        'height': 400,
        'content_weight': 1e5,
        'style_weight': 3e4,
        'tv_weight': 1e0,
        'optimizer': 'lbfgs',
        'model': 'vgg19',
        'init_method': 'content',
        'saving_freq': -1,
    }
    results_path = neural_style_transfer(optimization_config)
    
    
    

    
    

C:\Users\Saransh\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\Saransh\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Using vgg19 in the optimization procedure.
L-BFGS | iteration: 000, total loss=5120649068544.0000, content_loss=      0.0000, style loss=5120641920000.0000, tv loss=7284129.0000
L-BFGS | iteration: 001, total loss=5120647495680.0000, content_loss=      0.0002, style loss=5120640000000.0000, tv loss=7284129.0000
L-BFGS | iteration: 002, total loss=5120637534208.0000, content_loss=      0.0036, style loss=5120630400000.0000, tv loss=7284131.5000
L-BFGS | iteration: 003, total loss=5120627048448.0000, content_loss=      0.0127, style loss=5120619840000.0000, tv loss=7284135.0000
L-BFGS | iteration: 004, total loss=5120609222656.0000, content_loss=      0.0439, style loss=5120602080000.0000, tv loss=7284139.5000
L-BFGS | iteration: 005, total loss=5120428343296.0000, content_loss=      1.3218, style loss=5120421120000.0000, tv loss=7284189.0000
L-BFGS | iteration: 006, total loss=5118806720512.0000, content_loss=     92.0850, style loss=5118799200000.0000, tv loss=7284626.0000
L-BFGS | ite

L-BFGS | iteration: 060, total loss=108148957184.0000, content_loss=1426477441.4062, style loss=106705875000.0000, tv loss=16599554.0000
L-BFGS | iteration: 061, total loss=105293012992.0000, content_loss=1403206738.2812, style loss=103873365000.0000, tv loss=16440593.0000
L-BFGS | iteration: 062, total loss=102136119296.0000, content_loss=1445544921.8750, style loss=100673857500.0000, tv loss=16715941.0000
L-BFGS | iteration: 063, total loss=100022042624.0000, content_loss=1432174023.4375, style loss=98573212500.0000, tv loss=16653796.0000
L-BFGS | iteration: 064, total loss=98440830976.0000, content_loss=1420881347.6562, style loss=97003395000.0000, tv loss=16558906.0000
L-BFGS | iteration: 065, total loss=96089071616.0000, content_loss=1425015429.6875, style loss=94647405000.0000, tv loss=16657019.0000
L-BFGS | iteration: 066, total loss=92757123072.0000, content_loss=1440022363.2812, style loss=91300305000.0000, tv loss=16794326.0000
L-BFGS | iteration: 067, total loss=89872064512.

L-BFGS | iteration: 121, total loss=31491637248.0000, content_loss=1521342187.5000, style loss=29952438750.0000, tv loss=17855812.0000
L-BFGS | iteration: 122, total loss=31092289536.0000, content_loss=1523429589.8438, style loss=29551005000.0000, tv loss=17854896.0000
L-BFGS | iteration: 123, total loss=30859763712.0000, content_loss=1529813378.9062, style loss=29312019375.0000, tv loss=17931704.0000
L-BFGS | iteration: 124, total loss=30520205312.0000, content_loss=1527302539.0625, style loss=28975001250.0000, tv loss=17900794.0000
L-BFGS | iteration: 125, total loss=30194944000.0000, content_loss=1527428027.3438, style loss=28649613750.0000, tv loss=17903220.0000
L-BFGS | iteration: 126, total loss=29854111744.0000, content_loss=1526711230.4688, style loss=28309475625.0000, tv loss=17926260.0000
L-BFGS | iteration: 127, total loss=29485281280.0000, content_loss=1531430859.3750, style loss=27935904375.0000, tv loss=17946972.0000
L-BFGS | iteration: 128, total loss=29238495232.0000, c

L-BFGS | iteration: 182, total loss=18905886720.0000, content_loss=1571665527.3438, style loss=17315589375.0000, tv loss=18629740.0000
L-BFGS | iteration: 183, total loss=18769481728.0000, content_loss=1573055859.3750, style loss=17177771250.0000, tv loss=18655936.0000
L-BFGS | iteration: 184, total loss=18601109504.0000, content_loss=1574913378.9062, style loss=17007513750.0000, tv loss=18681232.0000
L-BFGS | iteration: 185, total loss=18473158656.0000, content_loss=1574555761.7188, style loss=16879908750.0000, tv loss=18694794.0000
L-BFGS | iteration: 186, total loss=18359158784.0000, content_loss=1575509765.6250, style loss=16764950625.0000, tv loss=18697324.0000
L-BFGS | iteration: 187, total loss=18397415424.0000, content_loss=1576849511.7188, style loss=16801816875.0000, tv loss=18750012.0000
L-BFGS | iteration: 188, total loss=18305243136.0000, content_loss=1576224707.0312, style loss=16710296250.0000, tv loss=18721332.0000
L-BFGS | iteration: 189, total loss=18233972736.0000, c

...

L-BFGS | iteration: 738, total loss=6793951744.0000, content_loss=1691110156.2500, style loss=5078416406.2500, tv loss=24425204.0000
L-BFGS | iteration: 739, total loss=6790038016.0000, content_loss=1691072070.3125, style loss=5074544062.5000, tv loss=24422052.0000
L-BFGS | iteration: 740, total loss=6786360320.0000, content_loss=1691070703.1250, style loss=5070864843.7500, tv loss=24424538.0000
L-BFGS | iteration: 741, total loss=6781932544.0000, content_loss=1691178125.0000, style loss=5066321250.0000, tv loss=24433380.0000
L-BFGS | iteration: 742, total loss=6776554496.0000, content_loss=1691269140.6250, style loss=5060839687.5000, tv loss=24445340.0000
L-BFGS | iteration: 743, total loss=6772055552.0000, content_loss=1691351171.8750, style loss=5056257187.5000, tv loss=24447548.0000
L-BFGS | iteration: 744, total loss=6769882624.0000, content_loss=1691238671.8750, style loss=5054190937.5000, tv loss=24452516.0000
L-BFGS | iteration: 745, total loss=6766706688.0000, content_loss=169

L-BFGS | iteration: 800, total loss=6571219968.0000, content_loss=1691685937.5000, style loss=4854823593.7500, tv loss=24710648.0000
L-BFGS | iteration: 801, total loss=6567507456.0000, content_loss=1691704882.8125, style loss=4851090000.0000, tv loss=24712556.0000
L-BFGS | iteration: 802, total loss=6563830272.0000, content_loss=1691653125.0000, style loss=4847460468.7500, tv loss=24716686.0000
L-BFGS | iteration: 803, total loss=6559950848.0000, content_loss=1691588476.5625, style loss=4843645312.5000, tv loss=24716886.0000
L-BFGS | iteration: 804, total loss=6556308992.0000, content_loss=1691696289.0625, style loss=4839885937.5000, tv loss=24727166.0000
L-BFGS | iteration: 805, total loss=6552164864.0000, content_loss=1691627148.4375, style loss=4835807812.5000, tv loss=24730016.0000
L-BFGS | iteration: 806, total loss=6549037056.0000, content_loss=1691706054.6875, style loss=4832597343.7500, tv loss=24733824.0000
L-BFGS | iteration: 807, total loss=6546785280.0000, content_loss=169

L-BFGS | iteration: 862, total loss=6378504192.0000, content_loss=1691269921.8750, style loss=4662275156.2500, tv loss=24958836.0000
L-BFGS | iteration: 863, total loss=6376735744.0000, content_loss=1691341406.2500, style loss=4660432500.0000, tv loss=24962144.0000
L-BFGS | iteration: 864, total loss=6374014976.0000, content_loss=1691315820.3125, style loss=4657732031.2500, tv loss=24967288.0000
L-BFGS | iteration: 865, total loss=6370599424.0000, content_loss=1691347460.9375, style loss=4654278750.0000, tv loss=24973318.0000
L-BFGS | iteration: 866, total loss=6366779904.0000, content_loss=1691231445.3125, style loss=4650573281.2500, tv loss=24974888.0000
L-BFGS | iteration: 867, total loss=6363661312.0000, content_loss=1691308203.1250, style loss=4647373593.7500, tv loss=24979712.0000
L-BFGS | iteration: 868, total loss=6361500672.0000, content_loss=1691202734.3750, style loss=4645315781.2500, tv loss=24982384.0000
L-BFGS | iteration: 869, total loss=6358569984.0000, content_loss=169

L-BFGS | iteration: 924, total loss=6218430976.0000, content_loss=1690536132.8125, style loss=4502723437.5000, tv loss=25171612.0000
L-BFGS | iteration: 925, total loss=6216201216.0000, content_loss=1690526562.5000, style loss=4500495937.5000, tv loss=25178752.0000
L-BFGS | iteration: 926, total loss=6213883392.0000, content_loss=1690580664.0625, style loss=4498125468.7500, tv loss=25177822.0000
L-BFGS | iteration: 927, total loss=6211954688.0000, content_loss=1690521484.3750, style loss=4496252343.7500, tv loss=25180596.0000
L-BFGS | iteration: 928, total loss=6209568256.0000, content_loss=1690556640.6250, style loss=4493826562.5000, tv loss=25184744.0000
L-BFGS | iteration: 929, total loss=6206930944.0000, content_loss=1690537304.6875, style loss=4491205312.5000, tv loss=25188504.0000
L-BFGS | iteration: 930, total loss=6203532800.0000, content_loss=1690592187.5000, style loss=4487747812.5000, tv loss=25193078.0000
L-BFGS | iteration: 931, total loss=6200941568.0000, content_loss=169

L-BFGS | iteration: 986, total loss=6076749824.0000, content_loss=1690326562.5000, style loss=4361048437.5000, tv loss=25374940.0000
L-BFGS | iteration: 987, total loss=6074877440.0000, content_loss=1690283203.1250, style loss=4359214687.5000, tv loss=25379268.0000
L-BFGS | iteration: 988, total loss=6072417280.0000, content_loss=1690360546.8750, style loss=4356672656.2500, tv loss=25383986.0000
L-BFGS | iteration: 989, total loss=6070405632.0000, content_loss=1690269335.9375, style loss=4354748906.2500, tv loss=25387736.0000
L-BFGS | iteration: 990, total loss=6068292608.0000, content_loss=1690318750.0000, style loss=4352585625.0000, tv loss=25387808.0000
L-BFGS | iteration: 991, total loss=6066656768.0000, content_loss=1690346484.3750, style loss=4350917343.7500, tv loss=25393378.0000
L-BFGS | iteration: 992, total loss=6064793600.0000, content_loss=1690290234.3750, style loss=4349110781.2500, tv loss=25392536.0000
L-BFGS | iteration: 993, total loss=6062228992.0000, content_loss=169